# Chapter 3.2: Context Providers

> **Previously**: You built Ask HR manually and felt the pain of boilerplate code. Now let's see how Context Providers make this elegant.

In this notebook, Ask HR finally gets easy. Context Providers automatically inject relevant documents into every request - no more manual search-and-stuff-the-prompt logic.

**What you'll build:**
1. Ask HR with automatic RAG (one config, zero boilerplate)
2. Multi-source retrieval (combine multiple knowledge bases)
3. Personalized responses (inject user profile, time, memory)

---

## Prerequisites

Complete `03.1-rag-fundamentals.ipynb` first (you need the search index).

In [ ]:
# Suppress Azure SDK warning FIRST before any imports
# (agent-framework uses k_nearest_neighbors but SDK expects k - harmless mismatch)
import logging
logging.getLogger("azure.search.documents").setLevel(logging.ERROR)

# Environment Setup & Validation
import sys
sys.path.insert(0, '..')

from workshop_utils import validate_env
validate_env()

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

# Azure AI Search configuration
search_endpoint = os.getenv("AZURE_SEARCH_ENDPOINT")
search_key = os.getenv("SEARCH_API_KEY")
index_name = os.getenv("INDEX_NAME")

# Azure OpenAI configuration (same as other notebooks)
openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
openai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
api_version = os.getenv("AZURE_OPENAI_API_VERSION")
chat_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
embedding_deployment = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME")

print(f"Search Endpoint: {search_endpoint}")
print(f"Index Name: {index_name}")
print(f"OpenAI Endpoint: {openai_endpoint}")

---

## Part 1: Understanding Context Providers

Context Providers are middleware that run before and after every agent invocation. Let's see one in action first, then understand how it works.

```
┌────────────────────────────────────────────────────────────────────┐
│                    Context Provider Pipeline                       │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│  User Message ──► invoking() ──► Agent ──► invoked() ──► Response │
│                      │                         │                   │
│                      ▼                         ▼                   │
│              Inject Context             Extract/Store Info         │
│              (from search)              (for future use)           │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

Two methods, two purposes:\n
- **`invoking()`** runs before the LLM - use it to inject context, instructions, or tools\n
- **`invoked()`** runs after the LLM - use it to extract info, update memory, or log\n
\n
This solves everything we struggled with in 03.1: no more boilerplate, built-in multi-turn support, automatic token management, and easy multi-source aggregation.

---

## Part 2: AzureAISearchContextProvider

The Agent Framework provides a built-in `AzureAISearchContextProvider` that automatically:

1. Searches your index on every user message
2. Injects relevant documents as context
3. Handles authentication and connection management

In [ ]:
from openai import AsyncAzureOpenAI
from agent_framework import ChatAgent
from agent_framework.azure import AzureOpenAIChatClient, AzureAISearchContextProvider


async def get_embedding(text: str) -> list[float]:
    """Generate embedding for search queries."""
    async with AsyncAzureOpenAI(
        api_version=api_version,
        azure_endpoint=openai_endpoint,
        api_key=openai_api_key,
    ) as client:
        response = await client.embeddings.create(
            input=text,
            model=embedding_deployment
        )
        return response.data[0].embedding


# Create chat client (AzureOpenAIChatClient doesn't require explicit cleanup)
chat_client = AzureOpenAIChatClient(
    endpoint=openai_endpoint,
    api_key=openai_api_key,
    api_version=api_version,
    deployment_name=chat_deployment,
)

# Create the context provider with embedding function for vector search
search_provider = AzureAISearchContextProvider(
    endpoint=search_endpoint,
    index_name=index_name,
    api_key=search_key,
    mode="semantic",
    top_k=3,
    semantic_configuration_name=f"{index_name}-semantic-configuration",
    vector_field_name="text_vector",
    embedding_function=get_embedding,
)

async with (
    search_provider,
    ChatAgent(
        chat_client=chat_client,
        name="HRAgent",
        instructions="""
        You are a helpful HR assistant for Contoso Electronics employees.
        
        Answer questions using ONLY the context provided to you.
        If the context doesn't contain the answer, say "I don't have that information."
        Be concise and cite specific plan names when applicable.
        """,
        context_provider=search_provider,
    ) as agent,
):
    response = await agent.run("What is the deductible for Northwind Health Plus?")
    print(response.text)


### Compare: Manual RAG vs. Context Provider

Notice the difference? In 03.1, we wrote ~30 lines of search logic, prompt construction, and context injection for every query. Here, it's ~5 lines of configuration. The provider automatically searches on every message, handles multi-turn conversations with `AgentThread`, and serialization is built-in. That's the point of abstractions.

### Streaming Responses

Context Providers work seamlessly with streaming:

In [ ]:
async with (
    search_provider,
    ChatAgent(
        chat_client=chat_client,
        name="HRAgent",
        instructions="You are a helpful HR assistant. Answer based on provided context.",
        context_provider=search_provider,
    ) as agent,
):
    print("Agent: ", end="", flush=True)
    
    async for chunk in agent.run_stream("What is covered under preventive care?"):
        if chunk.text:
            print(chunk.text, end="", flush=True)
    
    print()

### Multi-turn Conversations with AgentThread

Context Providers maintain state across conversation turns using `AgentThread`:

In [ ]:
async with (
    search_provider,
    ChatAgent(
        chat_client=chat_client,
        name="HRAgent",
        instructions="You are a helpful HR assistant. Answer based on provided context. Be concise.",
        context_provider=search_provider,
    ) as agent,
):
    # Create a conversation thread
    thread = agent.get_new_thread()
    
    # Multi-turn conversation
    questions = [
        "What is the deductible for Northwind Health Plus?",
        "And what about the coinsurance?",  # Follow-up referring to same plan
        "How does that compare to Northwind Standard?",  # Comparison
    ]
    
    for question in questions:
        print(f"\nUser: {question}")
        response = await agent.run(question, thread=thread)
        print(f"Agent: {response.text}")

---

## Part 3: Multiple Knowledge Sources with AggregateContextProvider

> What if Ask HR needs to answer from multiple knowledge bases - health plans AND company policies?

The `AggregateContextProvider` combines multiple providers by calling all of them in parallel and merging their results. Since `ChatAgent` accepts only one `context_provider`, we wrap multiple providers in an aggregate.

### Exercise 1: Build a Document Filter Provider

The search provider returns results from all documents. What if you want Ask HR to ONLY answer from the employee handbook (ignoring health plan docs)?

**Your task:** Create a `DocumentFilterContextProvider` that wraps a search provider and filters results by document title.

```python
class DocumentFilterContextProvider(ContextProvider):
    def __init__(self, search_provider, allowed_titles: list[str]):
        # TODO: Store the search provider and allowed titles
        pass
    
    async def invoking(self, messages, **kwargs) -> Context:
        # TODO: Call the search provider, then filter results
        # to only include documents with titles in allowed_titles
        pass
```

<details>
<summary>Click to see solution</summary>

```python
class DocumentFilterContextProvider(ContextProvider):
    def __init__(self, search_provider, allowed_titles: list[str]):
        self.search_provider = search_provider
        self.allowed_titles = [t.lower() for t in allowed_titles]
    
    async def invoking(self, messages, **kwargs) -> Context:
        # Get context from the underlying provider
        context = await self.search_provider.invoking(messages, **kwargs)
        
        # Filter instructions to only include allowed sources
        # (This is a simplified example - real filtering depends on
        # how your provider formats the context)
        if context.instructions:
            lines = context.instructions.split('\\n')
            filtered = [l for l in lines 
                       if any(t in l.lower() for t in self.allowed_titles)]
            context = Context(instructions='\\n'.join(filtered))
        
        return context
    
    async def __aenter__(self):
        await self.search_provider.__aenter__()
        return self
    
    async def __aexit__(self, *args):
        await self.search_provider.__aexit__(*args)

# Usage:
handbook_only = DocumentFilterContextProvider(
    search_provider, 
    allowed_titles=['employee_handbook.pdf']
)
```

</details>

In [ ]:
# Check if second index is configured
load_dotenv(override=True)
index2_name = os.getenv("INDEX_2_NAME")

if index2_name:
    print(f"✅ Second index configured: {index2_name}")
else:
    print("⚠️ INDEX2_NAME not found in .env - complete Exercise 1 first")
    print("   You can still proceed with a single index, but the multi-source examples won't work.")

### AggregateContextProvider Implementation

The `AggregateContextProvider` combines multiple context providers by:
1. Calling `invoking()` on all providers in parallel
2. Merging their returned contexts (instructions, messages, tools)
3. Calling `invoked()` on all providers after the agent responds

In [ ]:
import asyncio
from collections.abc import MutableSequence, Sequence
from contextlib import AsyncExitStack
from types import TracebackType
from typing import Any

from agent_framework import ContextProvider, Context, ChatMessage


class AggregateContextProvider(ContextProvider):
    """
    A ContextProvider that combines multiple context providers.
    
    It delegates events to all providers and aggregates their responses.
    This allows you to combine multiple knowledge sources into a single provider.
    """
    
    def __init__(self, providers: list[ContextProvider]):
        self.providers = providers
        self._exit_stack: AsyncExitStack | None = None
    
    async def invoking(
        self,
        messages: ChatMessage | MutableSequence[ChatMessage],
        **kwargs: Any
    ) -> Context:
        """Aggregate context from all providers."""
        # Call all providers in parallel
        contexts = await asyncio.gather(*[
            provider.invoking(messages, **kwargs) 
            for provider in self.providers
        ])
        
        # Merge all contexts
        instructions = ""
        all_messages: list[ChatMessage] = []
        all_tools: list = []
        
        for ctx in contexts:
            if ctx.instructions:
                instructions += ctx.instructions + "\n"
            if ctx.messages:
                all_messages.extend(ctx.messages)
            if ctx.tools:
                all_tools.extend(ctx.tools)
        
        return Context(
            instructions=instructions.strip() if instructions else None,
            messages=all_messages if all_messages else None,
            tools=all_tools if all_tools else None,
        )
    
    async def invoked(
        self,
        request_messages: ChatMessage | Sequence[ChatMessage],
        response_messages: ChatMessage | Sequence[ChatMessage] | None = None,
        invoke_exception: Exception | None = None,
        **kwargs: Any,
    ) -> None:
        """Notify all providers after invocation."""
        await asyncio.gather(*[
            provider.invoked(
                request_messages=request_messages,
                response_messages=response_messages,
                invoke_exception=invoke_exception,
                **kwargs,
            )
            for provider in self.providers
        ])
    
    async def __aenter__(self):
        """Enter all provider contexts."""
        self._exit_stack = AsyncExitStack()
        await self._exit_stack.__aenter__()
        
        for provider in self.providers:
            await self._exit_stack.enter_async_context(provider)
        
        return self
    
    async def __aexit__(
        self,
        exc_type: type[BaseException] | None,
        exc_val: BaseException | None,
        exc_tb: TracebackType | None,
    ) -> None:
        """Exit all provider contexts."""
        if self._exit_stack:
            await self._exit_stack.__aexit__(exc_type, exc_val, exc_tb)
            self._exit_stack = None


print("AggregateContextProvider defined ✅")

In [ ]:
# Skip this cell if INDEX2_NAME is not configured
if not index2_name:
    print("Skipping multi-source example - INDEX2_NAME not configured")
else:
    # Create individual providers for each knowledge source
    health_plans_provider = AzureAISearchContextProvider(
        endpoint=search_endpoint,
        index_name=index_name,
        api_key=search_key,
        mode="semantic",
        top_k=3,
        semantic_configuration_name=f"{index_name}-semantic-configuration",
        vector_field_name="text_vector",
        embedding_function=get_embedding,
    )

    general_hr_provider = AzureAISearchContextProvider(
        endpoint=search_endpoint,
        index_name=index2_name,
        api_key=search_key,
        mode="semantic",
        top_k=3,
        semantic_configuration_name=f"{index2_name}-semantic-configuration",
        vector_field_name="text_vector",
        embedding_function=get_embedding,
    )

    # Combine providers
    composite_provider = AggregateContextProvider([
        health_plans_provider,
        general_hr_provider,
    ])

    # Test questions that span both knowledge bases
    TEST_QUESTIONS = [
        "What's the PerksPlus program?",          # General HR (index2)
        "What is the deductible for Northwind Standard?",  # Health plans (index1)
        "What benefits are available to employees?",       # Spans both
    ]

    async with (
        composite_provider,
        ChatAgent(
            chat_client=chat_client,
            name="ComprehensiveHRAgent",
            instructions="""
            You are a helpful HR assistant for Contoso Electronics employees.

            You have access to two knowledge bases:
            1. **Health Plan Details**: Northwind Health Plus and Standard plan specifics
            2. **General HR Knowledge**: Company policies, benefits overview, PerksPlus

            Guidelines:
            - Answer using ONLY the context provided
            - Keep responses to 2-3 sentences
            - Cite specific sources when relevant
            """,
            context_provider=composite_provider,
        ) as agent,
    ):
        for question in TEST_QUESTIONS:
            print(f"\nUser: {question}")
            print("Agent: ", end="", flush=True)
            
            async for chunk in agent.run_stream(question):
                if chunk.text:
                    print(chunk.text, end="", flush=True)
            
            print()


---

## Part 4: Custom Context Providers

> Ask HR is working, but it treats every employee the same. Let's make it personal - it should know who it's talking to, and be aware of time-sensitive information like enrollment periods.

Custom Context Providers let you inject any context you want: user profiles, current time, extracted memories, or data from non-Azure sources.

In [ ]:
from datetime import datetime


class TimeAwareContextProvider(ContextProvider):
    """
    A context provider that injects the current date and time.
    
    Useful for agents that need to understand temporal context,
    like deadlines, enrollment periods, or time-sensitive policies.
    """
    
    async def invoking(
        self,
        messages: ChatMessage | MutableSequence[ChatMessage],
        **kwargs: Any
    ) -> Context:
        current_time = datetime.now().strftime("%A, %B %d, %Y at %I:%M %p")
        return Context(
            instructions=f"Current date and time: {current_time}"
        )


class UserProfileContextProvider(ContextProvider):
    """
    A context provider that injects user profile information.
    
    This allows personalized responses based on the user's
    enrollment status, plan selection, or employment details.
    """
    
    def __init__(self, user_profile: dict):
        self.user_profile = user_profile
    
    async def invoking(
        self,
        messages: ChatMessage | MutableSequence[ChatMessage],
        **kwargs: Any
    ) -> Context:
        profile_text = "\n".join([
            f"- {key}: {value}" 
            for key, value in self.user_profile.items()
        ])
        return Context(
            instructions=f"User Profile:\n{profile_text}"
        )


print("Custom context providers defined ✅")

In [ ]:
# Combine custom providers with search provider
user_profile = {
    "Name": "Alice Johnson",
    "Department": "Engineering",
    "Current Plan": "Northwind Health Plus",
    "Employment Status": "Full-time",
}

time_provider = TimeAwareContextProvider()
profile_provider = UserProfileContextProvider(user_profile)

# Create a search provider for knowledge retrieval
knowledge_provider = AzureAISearchContextProvider(
    endpoint=search_endpoint,
    index_name=index_name,
    api_key=search_key,
    mode="semantic",
    top_k=3,
    semantic_configuration_name=f"{index_name}-semantic-configuration",
    vector_field_name="text_vector",
    embedding_function=get_embedding,
)

# Combine all providers
combined_provider = AggregateContextProvider([
    time_provider,
    profile_provider,
    knowledge_provider,
])

async with (
    combined_provider,
    ChatAgent(
        chat_client=chat_client,
        name="PersonalizedHRAgent",
        instructions="""
        You are a personalized HR assistant.
        
        Use the user profile to personalize your responses.
        Reference their current plan when answering health questions.
        Be aware of the current date for any time-sensitive information.
        """,
        context_provider=combined_provider,
    ) as agent,
):
    # The agent now has access to time, user profile, AND search results
    response = await agent.run("What is my deductible?")
    print(f"Agent: {response.text}")


### Exercise 2: Open Enrollment Awareness

The `TimeAwareContextProvider` just injects the date. But what if Ask HR should **proactively mention** when open enrollment is happening?

**Your task:** Create an `EnrollmentAwareProvider` that:
1. Checks if the current date is within open enrollment period (Nov 1 - Nov 30)
2. If so, searches for enrollment-related documents and injects them as additional context
3. Adds an instruction reminding the agent to mention the enrollment deadline

```python
class EnrollmentAwareProvider(ContextProvider):
    def __init__(self, search_provider):
        self.search_provider = search_provider
        self.enrollment_start = (11, 1)  # November 1
        self.enrollment_end = (11, 30)   # November 30
    
    async def invoking(self, messages, **kwargs) -> Context:
        now = datetime.now()
        # TODO: Check if we're in enrollment period
        # TODO: If yes, search for enrollment docs and add reminder
        # TODO: If no, return empty context
        pass
```

**Test it:** When your provider detects enrollment season, the agent should mention the deadline even for unrelated questions like \"What's my deductible?\"

<details>
<summary>Click to see solution</summary>

```python
class EnrollmentAwareProvider(ContextProvider):
    def __init__(self, search_provider):
        self.search_provider = search_provider
        self.enrollment_start = (11, 1)
        self.enrollment_end = (11, 30)
    
    def _is_enrollment_period(self) -> bool:
        now = datetime.now()
        return (self.enrollment_start <= (now.month, now.day) <= self.enrollment_end)
    
    async def invoking(self, messages, **kwargs) -> Context:
        if not self._is_enrollment_period():
            return Context()
        
        # During enrollment, always inject enrollment context
        # Force a search for enrollment info
        enrollment_query = ChatMessage.user(\"open enrollment deadline benefits changes\")
        enrollment_context = await self.search_provider.invoking([enrollment_query], **kwargs)
        
        reminder = \"\"\"\nIMPORTANT: Open enrollment is currently active!
        Deadline: November 30th. Remind users to review their benefits choices.\"\"\"
        
        instructions = (enrollment_context.instructions or \"\") + reminder
        return Context(instructions=instructions)
    
    async def __aenter__(self):
        await self.search_provider.__aenter__()
        return self
    
    async def __aexit__(self, *args):
        await self.search_provider.__aexit__(*args)
```

</details>

### Exercise 3: Build a Memory Extraction Provider

Create a custom context provider that:
1. In `invoked()`, extracts any facts the user mentions (like preferences)
2. In `invoking()`, injects those stored facts as context

This creates a simple long-term memory for the agent.

<details>
<summary>Click to see solution</summary>

```python
class SimpleMemoryProvider(ContextProvider):
    """A context provider that extracts and remembers user facts."""
    
    def __init__(self):
        self.memories: list[str] = []
    
    async def invoking(
        self,
        messages: ChatMessage | MutableSequence[ChatMessage],
        **kwargs: Any
    ) -> Context:
        if not self.memories:
            return Context()
        
        memory_text = "\n".join([f"- {m}" for m in self.memories])
        return Context(
            instructions=f"Remembered facts about the user:\n{memory_text}"
        )
    
    async def invoked(
        self,
        request_messages: ChatMessage | Sequence[ChatMessage],
        response_messages: ChatMessage | Sequence[ChatMessage] | None = None,
        **kwargs: Any,
    ) -> None:
        # Simple extraction: look for "I prefer" or "I like" statements
        msgs = [request_messages] if isinstance(request_messages, ChatMessage) else list(request_messages)
        
        for msg in msgs:
            content = str(msg.content).lower()
            if "i prefer" in content or "i like" in content:
                self.memories.append(str(msg.content))
```

</details>

---

## Summary

**Ask HR is now production-ready.** You started with 30+ lines of manual RAG boilerplate in 03.1, and now you have a personalized, time-aware HR assistant in ~10 lines of configuration.

The journey:
1. **Built-in RAG** - `AzureAISearchContextProvider` handles search automatically
2. **Multi-source knowledge** - `AggregateContextProvider` combines health plans + company policies
3. **Personalization** - Custom providers inject user profiles, time, and memories

The key insight: Context Providers are middleware. They run `invoking()` before the LLM sees your message (to inject context) and `invoked()` after (to extract and store information). This pattern - inject before, extract after - is the foundation of intelligent agents.

### What's Next

In Chapter 4, you'll give Ask HR the ability to **take action** - not just answer questions, but actually help employees change their benefits, submit requests, and interact with external systems.

### Going Deeper

The Agent Framework has additional context providers we didn't cover: Mem0 for long-term memory, Redis for fast retrieval, and agentic search for multi-hop reasoning. See the [samples directory](https://github.com/microsoft/agent-framework/tree/main/python/samples/getting_started/context_providers) for examples.